# Calculate mangrove blue carbon
Based on [ee code](https://code.earthengine.google.com/b1e2d678541dc3e9ff46eaf85df175c9)

The export grid used has been [calculated](../download-grid-creation.ipynb) as 
the max allowed pixel size for the GEE export: 1e9 pixels

This code creates a raster imageCollection representing the total carbon
(in CO2e units) stock of global mangrove ecotypes per hectare (t CO2e / ha).  

Input data:  
* soc: mangrove soil organic carbon t OC / m2
* agb: above ground biomass t / m2

Conversions:
* soc: t OC / m2 -> t CO2e / m2  
* agb: t / m2 -> t CO2e / m2
* bgb: agb t / m2 -> bgb  t / m2 -> t CO2e / m2

Factors:
* Above to Below Ground Biomass: 0.49 (Simard et al. 2019)
* Biomass to Carbon: 0.451 (Simard et al. 2019)
* Organic Carbon to CO2e: 11/3 = 3.66 

In [26]:
import os
from pathlib import Path
import json
import ee
import geemap
import geopandas as gpd
import shapely.speedups
shapely.speedups.enable()

In [2]:
# Trigger the authentication flow.
#ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate()
# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


### Data Assets

In [32]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/datasets/raw'

# @TODO: Add expected data files source as an environment variable.
assert BASE_DIR == '/home/jovyan/work/datasets/raw', f'{BASE_DIR} is not the correct directory'

# variables
grid_filename = Path(f'{BASE_DIR}/grid_test.shp')
base_gcs_url = 'https://storage.googleapis.com/mangrove_atlas/ee_export_tiffs'
dataset = 'mangrove_blue_carbon'

In [ ]:
## Assets selection
#agb: above-ground biomass density
agb = ee.Image(ee.ImageCollection(
    "projects/global-mangrove-watch/mangrove-properties/mangrove_aboveground_biomass_1996-2016").first())
# oc : soil organic carbon
soc = ee.Image("projects/global-mangrove-watch/mangrove-properties/mangroves_SOC30m_0_100cm")

### Create the Blue carbon asset

In [34]:
export_tiles = gpd.read_file(grid_filename)
export_tiles['url'] = export_tiles.apply(lambda row: f'{base_gcs_url}/{dataset}/<year>/{row.id}.tif', axis=1)

In [195]:
fc = geemap.geopandas_to_ee(export_tiles)

In [175]:
# SET CONSTANTS
# factor to convert above-ground biomass to below-ground biomass (Simard et al. 2019)
agb_to_bgb = ee.Number(0.49)
# factor to convert above-ground biomass to organic carbon (Simard et al. 2019)
biomass_to_OC = ee.Number(0.451)
# factor to convert organic carbon to CO2 equivalent
OC_to_CO2e = ee.Number(11).divide(3)

maxPixels = 1e12

nms = soc.projection().nominalScale().getInfo()

In [159]:
def organic_c_to_co2(oc: ee.Image) -> ee.Image:
    '''
    Convert organic carbon to CO2 equivalent
    '''
    return oc.multiply(OC_to_CO2e)

def biomass_to_co2(biomass: ee.Image) -> ee.Image:
    '''
    Convert above-ground biomass to CO2 equivalent
    '''
    return organic_c_to_co2(biomass.multiply(biomass_to_OC))

def above_to_below(agb: ee.Image) -> ee.Image:
    '''
    Convert above-ground biomass to below-ground biomass
    '''
    return agb.multiply(agb_to_bgb)


In [160]:
def calculate_blue_carbon(soc: ee.Image, abg: ee.Image):
    """
    Calculate the blue carbon from soil organic carbon and the above ground biomass.
    """
    bgb = above_to_below(abg)
    soc_co2 = (organic_c_to_co2(soc.updateMask(soc.gt(0)))).rename('soc_co2e')
    abg_co2 = (biomass_to_co2(abg.updateMask(abg.gt(0)))).rename('agb_co2e')
    bgb_co2 = (biomass_to_co2(bgb)).rename('bgb_co2e')
    
    # Return the image with the blue carbon.
    return ee.Image(soc_co2.add(abg_co2).add(bgb_co2)).rename('total_co2e')

In [161]:
def exportDataTasks(asset: ee.ImageCollection, asset_name: str, gcbucket: str, geometry_collection: gpd.GeoDataFrame):
    """
    Export the data to GEE.

    Parameters
    ----------
    asset : ee.Image
        The image to export.
    asset_name : str
        The name of the asset.
    gcbucket : str
        The name of the GCS bucket.
    geometry_collection : ee.GeometryCollection
        The geometry collection to export the data.
    
    Returns
    -------
    List of tasks
    """
    fc = geemap.geopandas_to_ee(geometry_collection)

    nms = asset.projection().nominalScale().getInfo()
    years = asset.date().getInfo()['year']
    taskList = []
    
    # TODO: improve this loop
    
    for geometry_id in geometry_collection:
        for year in years:
            task = ee.batch.Export.image.toCloudStorage(
                image = ee.Image(asset.filterMetadata('year', 'equals', year)).clip(
                    fc.filterMetadata('id', 'equals', geometry_id).geometry()),
                bucket =gcbucket,
                fileNamePrefix = f'ee_export_tiffs/{asset_name}/{year}/{geometry_id}',
                description = f'{asset_name}_{year}_{geometry_id}',
                scale = nms,
                region = fc.filterMetadata('id', 'equals', geometry_id).geometry(),
                fileFormat = 'GeoTIFF', 
                maxPixels = 1e13,
                formatOptions = {'cloudOptimized': True})
            taskList.append(task)
    
    return taskList

In [ ]:
def batchExecute(taskList: list, batch_size: int = 20):
    """
    Execute the tasks in the list in baches of 20 as is the max allowed by GEE. 
    each task takes about 10 min to execute.
    """
    n_tasks = len(taskList)
    n_batches = Math.ceil(n_tasks / batch_size)
    
    for i in range(n_batches):
        for task in taskList[i*batch_size:(i+1)*batch_size]:
            task.start()

    return taskList

In [162]:
blue_carbon = calculate_blue_carbon(soc, agb)

In [163]:
blue_carbon.projection().nominalScale().getInfo()

27.829872698318393

In [164]:
# data = blue_carbon.reduceRegions(
#     collection=fc,
#     reducer=ee.Reducer.sum()
# ).getInfo()

In [ ]:
# This is how we can dinamically create a url for downloading the data. there are some constrains on the nº of pixels that can be exported.
print(blue_carbon.getDownloadUrl({
    'scale': nms,
    'crs': 'EPSG:4326',
    'region': '[[39.184112548828125,-6.525458832948283],[39.494476318359375,-6.525458832948283],[39.494476318359375,-6.255236941610093],[39.184112548828125,-6.255236941610093],[39.184112548828125,-6.525458832948283]]'
}))

: 

In [201]:
Map = geemap.Map(center=(-5, 39), zoom=12, basemap='HYBRID')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(blue_carbon, vis_params, 'SRTM DEM', True, 0.5)
Map.addLayer(fc, {}, "geopandas to ee")
Map

Map(center=[-5, 39], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [209]:
#dimensions = '1000000000000x1000000000000',
#skipEmptyTiles = True,
# a max of 20 at a time
task = ee.batch.Export.image.toCloudStorage(
           image = blue_carbon.clip(fc.filterMetadata('id', 'equals', 4).geometry()),
           bucket ='mangrove_atlas',
           fileNamePrefix = f'ee_export_tiffs/test_total_organic_carbon-3',
           description = 'test_total_organic_carbon-2',
           scale = nms,
           region = fc.filterMetadata('id', 'equals', 4).geometry(),
           fileFormat = 'GeoTIFF', 
           maxPixels = 1e13,
           formatOptions = {'cloudOptimized': True})
task.start()
task.id

'R3DQ6J4GYDBPF35CIOO5C6FB'

In [210]:
task.status()

{'state': 'READY',
 'description': 'test_total_organic_carbon-2',
 'creation_timestamp_ms': 1658840921125,
 'update_timestamp_ms': 1658840921125,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'R3DQ6J4GYDBPF35CIOO5C6FB',
 'name': 'projects/earthengine-legacy/operations/R3DQ6J4GYDBPF35CIOO5C6FB'}